# Using an example MEDS tool, ACES for labeling

In [ ]:
#@title Install ACES


!pip install es-aces

In [ ]:
# From the ACES documentation

task_config = """
description: >-
  This file specifies the base configuration for the prediction of a hospital los being greater than 3days,
  leveraging only the first 48 hours of data after admission, with a 24 hour gap between the input window
  and the target window. Patients who die or are discharged in the gap window are excluded. Note that this
  task is in-**hospital** los, not in-**ICU** los which is a different task.

predicates:
  hospital_admission:
    code: {regex: "HOSPITAL_ADMISSION//.*"}
  hospital_discharge:
    code: {regex: "HOSPITAL_DISCHARGE//.*"}
  death:
    code: MEDS_DEATH
  discharge_or_death:
    expr: or(hospital_discharge, death)

trigger: hospital_admission

windows:
  input:
    start: NULL
    end: trigger + 48h
    start_inclusive: True
    end_inclusive: True
    index_timestamp: end
  gap:
    start: input.end
    end: start + 24h
    start_inclusive: False
    end_inclusive: True
    has:
      hospital_admission: (None, 0)
      discharge_or_death: (None, 0)
  target:
    start: trigger
    end: start + 3d
    start_inclusive: False
    end_inclusive: True
    label: discharge_or_death
"""
!mkdir /content/tasks/ -p
TASK_NAME = "in_hospital_3d_los_after_48h"
TASK_CONFIG_FP = f"/content/tasks/{TASK_NAME}.yaml"
with open(TASK_CONFIG_FP, 'w') as f:
    f.write(task_config)

In [ ]:
!pip install es-aces

In [ ]:
!aces-cli --multirun data=sharded data.standard=meds data.root="$MIMICIV_MEDS_DIR/data" "data.shard=$(expand_shards  /content/meds/data/)" cohort_dir=" /content/tasks" cohort_name="$TASK_NAME" config_path="$TASK_CONFIG_FP"

In [ ]:
import polars as pl

# execute query and get results
df = pl.read_parquet(f"/content/tasks/{TASK_NAME}/**/*.parquet")

print("train prevalence: " + str(round(pl.read_parquet(f"/content/tasks/{TASK_NAME}/train/*.parquet")['boolean_value'].mean(), 3)))
print("tuning prevalence: " + str(round(pl.read_parquet(f"/content/tasks/{TASK_NAME}/tuning/*.parquet")['boolean_value'].mean(), 3)))
print("held_out prevalence: " + str(round(pl.read_parquet(f"/content/tasks/{TASK_NAME}/held_out/*.parquet")['boolean_value'].mean(), 3)))


df.sort('boolean_value')

In [ ]:
#@title Install meds-tab

!pip uninstall es-aces -y
!pip install meds-tab

In [ ]:
MIMICIV_MEDS_DIR = "/content/meds/"
OUTPUT_TABULARIZATION_DIR="/content/tabularized/"
TASK_DIR="/content/tasks/"
TASK_NAME="in_hospital_3d_los_after_48h"
OUTPUT_MODEL_DIR="/content/output/meds_tab/"

In [ ]:
!meds-tab-describe input_dir={MIMICIV_MEDS_DIR}/data output_dir={OUTPUT_TABULARIZATION_DIR}

In [ ]:
# Define the window sizes and aggregations to generate features for
WINDOW_SIZES = "tabularization.window_sizes=[1d,30d,365d]"
AGGREGATIONS = "tabularization.aggs=[static/present,code/count,value/count,value/sum,value/sum_sqd,value/min,value/max]"

In [ ]:
!rm -rf /content/tabularized/tabularize/

In [ ]:
!meds-tab-tabularize-static "input_dir=$MIMICIV_MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False $WINDOW_SIZES $AGGREGATIONS

In [ ]:
!meds-tab-tabularize-time-series --multirun "worker=range(0,2)" "hydra/launcher=joblib" "input_dir=$MIMICIV_MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False $WINDOW_SIZES $AGGREGATIONS

In [ ]:
!meds-tab-cache-task "input_dir={MIMICIV_MEDS_DIR}/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "input_label_dir=$TASK_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False $WINDOW_SIZES $AGGREGATIONS

In [ ]:
!meds-tab-xgboost --multirun "input_dir=$MIMICIV_MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "output_model_dir=$OUTPUT_MODEL_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False "hydra.sweeper.n_trials=10" $WINDOW_SIZES $AGGREGATIONS "tabularization.min_code_inclusion_count=10"